<a href="https://colab.research.google.com/github/josejailson/wide-deep/blob/main/wide%26deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Wide & Deep arquitecture

# A brief example on California Housing dataset

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing


housing = fetch_california_housing()


In [2]:
X = housing.data
y = housing.target

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [8]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

In [9]:
model = tf.keras.Model(inputs=[input_], outputs=[output])

In [10]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              metrics =[tf.keras.metrics.RootMeanSquaredError()],
              optimizer=tf.keras.optimizers.Adam())

In [11]:
normalization_layer.adapt(X_train)

In [12]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
516/516 [==============================] - 2s 3ms/step - loss: 1.2319 - root_mean_squared_error: 1.1099 - val_loss: 0.4939 - val_root_mean_squared_error: 0.7027
Epoch 2/20
516/516 [==============================] - 2s 4ms/step - loss: 0.4281 - root_mean_squared_error: 0.6543 - val_loss: 0.4238 - val_root_mean_squared_error: 0.6510
Epoch 3/20
516/516 [==============================] - 2s 3ms/step - loss: 0.3778 - root_mean_squared_error: 0.6147 - val_loss: 0.4402 - val_root_mean_squared_error: 0.6635
Epoch 4/20
516/516 [==============================] - 2s 3ms/step - loss: 0.3707 - root_mean_squared_error: 0.6089 - val_loss: 0.3840 - val_root_mean_squared_error: 0.6197
Epoch 5/20
516/516 [==============================] - 1s 2ms/step - loss: 0.3577 - root_mean_squared_error: 0.5981 - val_loss: 0.4044 - val_root_mean_squared_error: 0.6359
Epoch 6/20
516/516 [==============================] - 1s 2ms/step - loss: 0.3416 - root_mean_squared_error: 0.5845 - val_loss: 0.4328 - val_

# And if different features data subsets flow to each path?

In [13]:
import tensorflow as tf


input_wide = tf.keras.layers.Input(shape=[5], name="input_wide")
input_deep = tf.keras.layers.Input(shape=[6], name="input_deep")
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse",
              optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

In [15]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]


In [16]:
X_train_wide.shape

(16512, 5)

In [17]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)


In [18]:
history = model.fit({"input_wide": X_train_wide, "input_deep": X_train_deep},
                    y_train,
                    epochs=20,
                    validation_data=((X_test_wide, X_test_deep), y_test))


Epoch 1/20
516/516 [==============================] - 4s 5ms/step - loss: 1.2413 - root_mean_squared_error: 1.1141 - val_loss: 0.5902 - val_root_mean_squared_error: 0.7683
Epoch 2/20
516/516 [==============================] - 1s 2ms/step - loss: 0.5024 - root_mean_squared_error: 0.7088 - val_loss: 0.4510 - val_root_mean_squared_error: 0.6716
Epoch 3/20
516/516 [==============================] - 1s 2ms/step - loss: 0.4297 - root_mean_squared_error: 0.6555 - val_loss: 0.4175 - val_root_mean_squared_error: 0.6462
Epoch 4/20
516/516 [==============================] - 1s 2ms/step - loss: 0.3984 - root_mean_squared_error: 0.6312 - val_loss: 0.3887 - val_root_mean_squared_error: 0.6235
Epoch 5/20
516/516 [==============================] - 1s 2ms/step - loss: 0.3887 - root_mean_squared_error: 0.6235 - val_loss: 0.3887 - val_root_mean_squared_error: 0.6235
Epoch 6/20
516/516 [==============================] - 1s 2ms/step - loss: 0.3788 - root_mean_squared_error: 0.6154 - val_loss: 0.3735 - val_

In [11]:
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

1/1 [==============================] - 0s 214ms/step


## Adding a auxiliary output

In [24]:
input_wide = tf.keras.layers.Input(shape=[5], name="input_wide")
input_deep = tf.keras.layers.Input(shape=[6], name="input_deep")
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"),
              loss_weights=(0.9, 0.1),
              optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

In [26]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep),
    (y_train, y_train),
    epochs=20,
    validation_data=((X_test_wide, X_test_deep), (y_test, y_test))
)

Epoch 1/20
516/516 [==============================] - 3s 3ms/step - loss: 1.3006 - dense_16_loss: 1.2710 - dense_17_loss: 1.5674 - dense_16_root_mean_squared_error: 1.1274 - dense_17_root_mean_squared_error: 1.2519 - val_loss: 0.5755 - val_dense_16_loss: 0.5516 - val_dense_17_loss: 0.7907 - val_dense_16_root_mean_squared_error: 0.7427 - val_dense_17_root_mean_squared_error: 0.8892
Epoch 2/20
516/516 [==============================] - 1s 2ms/step - loss: 0.4973 - dense_16_loss: 0.4758 - dense_17_loss: 0.6908 - dense_16_root_mean_squared_error: 0.6898 - dense_17_root_mean_squared_error: 0.8312 - val_loss: 0.4455 - val_dense_16_loss: 0.4281 - val_dense_17_loss: 0.6024 - val_dense_16_root_mean_squared_error: 0.6543 - val_dense_17_root_mean_squared_error: 0.7762
Epoch 3/20
516/516 [==============================] - 1s 2ms/step - loss: 0.4518 - dense_16_loss: 0.4359 - dense_17_loss: 0.5950 - dense_16_root_mean_squared_error: 0.6602 - dense_17_root_mean_squared_error: 0.7713 - val_loss: 0.418

In [27]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

129/129 [==============================] - 0s 2ms/step - loss: 0.3391 - dense_16_loss: 0.3332 - dense_17_loss: 0.3917 - dense_16_root_mean_squared_error: 0.5773 - dense_17_root_mean_squared_error: 0.6259


In [31]:
weighted_sum_of_losses

0.3390950560569763